# Holder Wallet Conformance Flows (v3) - CTWalletCrossDeferred

# 0.0 Initial setup

## 0.1 Setup conformance

In [25]:
import uuid
import asyncio
from rich.console import Console

console = Console()

loop = asyncio.get_event_loop()

## 0.2 Create did:key:jwk_jcs-pub identifier using ES256 algorithm

In [26]:
from ebsi_wallet.did_key import KeyDid, PublicKeyJWK
import uuid

# generate crypto seed
crypto_seed = b'helloworld'

key_did = KeyDid(seed=crypto_seed)

# generate keypair
key_did.create_keypair()

# create public key jwk
public_key_jwk = PublicKeyJWK(
    kty=key_did.public_key_jwk['kty'],
    crv=key_did.public_key_jwk['crv'],
    x=key_did.public_key_jwk['x'],
    y=key_did.public_key_jwk['y']
)

# generate did
key_did.generate_did(public_key_jwk)

print("Decentralised identifier: ", key_did._did)

Decentralised identifier:  did:key:z2dmzD81cgPx8Vki7JbuuMmFYrWPgYoytykUZ3eyqht1j9KbptQwoaj2VP1S6Ahzo7REFCT4NBTPYdQinCZbCcyoqWKi9Q2uEW36DNSXhCwiYnGz6BAZkzytQAEBE5cPidCGnadH4SsLDbSZeG2SEChrqvQpdK4Mk8H32vs3B5g8Wr7kcc


## 1.1 Initiate Credential Issuance

In [27]:
from ebsi_wallet.util import parse_query_string_parameters_from_url
from ebsi_wallet.siop_auth.util import (
    accept_and_fetch_credential_offer, 
    fetch_openid_credential_issuer_configuration,
    fetch_openid_auth_server_configuration,
    fetch_credential_offer,
    CredentialTypes
)

credential_type =  CredentialTypes.CTWalletCrossDeferred
qr_code_data = await fetch_credential_offer(key_did._did, credential_type)
qr_code_data = qr_code_data.replace("openid-credential-offer://", "")
credential_offer_uri = parse_query_string_parameters_from_url(
    qr_code_data).get("credential_offer_uri")[0]
console.log("Credential offer URI: ", credential_offer_uri)

credential_offer = await accept_and_fetch_credential_offer(credential_offer_uri)
console.log("Credential offer: ", credential_offer)

credential_issuer_configuration = await fetch_openid_credential_issuer_configuration(credential_offer.credential_issuer)
console.log("Credential issuer configuration: ", credential_issuer_configuration)

auth_server_configuration = await fetch_openid_auth_server_configuration(credential_issuer_configuration.authorization_server)
console.log("Authorization server configuration: ", auth_server_configuration)


[23:27:29] Credential offer URI:                                                                   ]8;id=101830;file:///tmp/ipykernel_59965/3425914532.py\3425914532.py]8;;\:]8;id=114527;file:///tmp/ipykernel_59965/3425914532.py#15\15]8;;\
           https://api-conformance.ebsi.eu/conformance/v3/issuer-mock/offers/18fb6f8f-daa8-4b50-82                 
           a3-69cdc9defd5a                                                                                         

[23:27:30] Credential offer:                                                                       ]8;id=628540;file:///tmp/ipykernel_59965/3425914532.py\3425914532.py]8;;\:]8;id=844301;file:///tmp/ipykernel_59965/3425914532.py#18\18]8;;\
           CredentialOffer(                                                                                        
               credential_issuer='https://api-conformance.ebsi.eu/conformance/v3/issuer-mock',                     
               credentials=[                                                                                       
                   {                                                                                               
                       'format': 'jwt_vc',                                                                         
                       'types': [                                                                                  
                           'VerifiableCredential',                                                                 
                           'VerifiableAttestation',                                                                
                           'CTWalletCrossDeferred'                                                                 
                       ],                                                                                          
                       'trust_framework': {                                                                        
                           'name': 'ebsi',                                                                         
                           'type': 'Accreditation',                                                                
                           'uri': 'TIR link towards accreditation'                                                 
                       }                                                                                           
                   }                                                                                               
               ],                                                                                                  
               grants={                                                                                            
                   'authorization_code': {                                                                         
                       'issuer_state':                                                                             
           'eyJ0eXAiOiJKV1QiLCJhbGciOiJFUzI1NiIsImtpZCI6InVPa3Y1TE9pbzlYQnE2a1JRaHNaMHpOTVZ1czBjNF                 
           ZqcFB0cDY1dTVjWXcifQ.eyJjbGllbnRfaWQiOiJkaWQ6a2V5OnoyZG16RDgxY2dQeDhWa2k3SmJ1dU1tRllyV1                 
           BnWW95dHlrVVozZXlxaHQxajlLYnB0UXdvYWoyVlAxUzZBaHpvN1JFRkNUNE5CVFBZZFFpbkNaYkNjeW9xV0tpO                 
           VEydUVXMzZETlNYaEN3aVluR3o2QkFaa3p5dFFBRUJFNWNQaWRDR25hZEg0U3NMRGJTWmVHMlNFQ2hycXZRcGRL                 
           NE1rOEgzMnZzM0I1ZzhXcjdrY2MiLCJjcmVkZW50aWFsX3R5cGVzIjpbIlZlcmlmaWFibGVDcmVkZW50aWFsIiw                 
           iVmVyaWZpYWJsZUF0dGVzdGF0aW9uIiwiQ1RXYWxsZXRDcm9zc0RlZmVycmVkIl0sImlhdCI6MTY4NzE5NzQ0OS                 
           wiZXhwIjoxNjg3MTk3NzQ5LCJpc3MiOiJodHRwczovL2FwaS1jb25mb3JtYW5jZS5lYnNpLmV1L2NvbmZvcm1hb                 
           mNlL3YzL2lzc3Vlci1tb2NrIiwiYXVkIjoiaHR0cHM6Ly9hcGktY29uZm9ybWFuY2UuZWJzaS5ldS9jb25mb3Jt                 
           YW5jZS92My9hdXRoLW1vY2siLCJzdWIiOiJkaWQ6a2V5OnoyZG16RDgxY2dQeDhWa2k3SmJ1dU1tRllyV1BnWW9                 
           5dHlrVVozZXlxaHQxajlLYnB0UXdvYWoyVlAxUzZBaHpvN1JFRkNUNE5CVFBZZFFpbkNaYkNjeW9xV0tpOVEydU                 
           VXMzZETlNYaEN3aVluR3o2QkFaa3p5dFFBRUJFNWNQaWRDR25hZEg0U3NMRGJTWmVHMlNFQ2hycXZRcGRLNE1rO                 
           EgzMnZzM0I1ZzhXcjdrY2MifQ.6cwwXiWnAv7F8pU_i9XaPEzx1V-EOyJkp3TMi2ALlb_S8qFTGmfG91yZTovgQ                 
           5p96naDFovpeZSWpaJZSpHjFw'    

[23:27:31] Credential issuer configuration:                                                        ]8;id=713231;file:///tmp/ipykernel_59965/3425914532.py\3425914532.py]8;;\:]8;id=43191;file:///tmp/ipykernel_59965/3425914532.py#21\21]8;;\
           OpenIDCredentialIssuerConfig(                                                                           
               credential_issuer='https://api-conformance.ebsi.eu/conformance/v3/issuer-mock',                     
               authorization_server='https://api-conformance.ebsi.eu/conformance/v3/auth-mock',                    
               credential_endpoint='https://api-conformance.ebsi.eu/conformance/v3/issuer-mock/cre                 
           dential',                                                                                               
               deferred_credential_endpoint='https://api-conformance.ebsi.eu/conformance/v3/issuer                 
           -mock/credential_deferred',                                                                             
               credentials_supported=[                                                                             
                   {                                                                                               
                       'format': 'jwt_vc',                                                                         
                       'types': [                                                                                  
                           'VerifiableCredential',                                                                 
                           'VerifiableAttestation',                                                                
                           'VerifiableAuthorisationToOnboard'                                                      
                       ],                                                                                          
                       'trust_framework': {                                                                        
                           'name': 'ebsi',                                                                         
                           'type': 'Accreditation',                                                                
                           'uri': 'TIR link towards accreditation'                                                 
                       },                                                                                          
                       'display': [                                                                                
                           {'name': 'Verifiable Authorisation to onboard', 'locale': 'en-GB'}                      
                       ]                                                                                           
                   },                                                                                              
                   {                                                                                               
                       'format': 'jwt_vc',                                                                         
                       'types': [                                                                                  
                           'VerifiableCredential',                                                                 
                           'VerifiableAttestation',                                                                
                           'VerifiableAccreditation',                                                              
                           'VerifiableAccreditationToAttest'                                                       
                       ],                                                                                          
                       'trust_framework': 

           Authorization server configuration:                                                     ]8;id=756233;file:///tmp/ipykernel_59965/3425914532.py\3425914532.py]8;;\:]8;id=366096;file:///tmp/ipykernel_59965/3425914532.py#24\24]8;;\
           OpenIDAuthServerConfig(                                                                                 
               redirect_uris=[                                                                                     
                   'https://api-conformance.ebsi.eu/conformance/v3/auth-mock/direct_post'                          
               ],                                                                                                  
               issuer='https://api-conformance.ebsi.eu/conformance/v3/auth-mock',                                  
               authorization_endpoint='https://api-conformance.ebsi.eu/conformance/v3/auth-mock/au                 
           thorize',                                                                                               
               token_endpoint='https://api-conformance.ebsi.eu/conformance/v3/auth-mock/token',                    
               jwks_uri='https://api-conformance.ebsi.eu/conformance/v3/auth-mock/jwks',                           
               scopes_supported=['openid'],                                                                        
               response_types_supported=['vp_token', 'id_token'],                                                  
               response_modes_supported=['query'],                                                                 
               grant_types_supported=['authorization_code'],                                                       
               subject_types_supported=['public'],                                                                 
               id_token_signing_alg_values_supported=['ES256'],                                                    
               request_object_signing_alg_values_supported=['ES256'],                                              
               request_parameter_supported=True,                                                                   
               request_uri_parameter_supported=True,                                                               
               token_endpoint_auth_methods_supported=['private_key_jwt'],                                          
               request_authentication_methods_supported={                                                          
                   'authorization_endpoint': ['request_object']                                                    
               },                                                                                                  
               vp_formats_supported={                                                                              
                   'jwt_vp': {'alg_values_supported': ['ES256']},                                                  
                   'jwt_vc': {'alg_values_supported': ['ES256']}                                                   
               },                                                                                                  
               subject_syntax_types_supported=['did:key', 'did:ebsi'],                                             
               subject_syntax_types_discriminations=['did:key:jwk_jcs-pub', 'did:ebsi:v1'],                        
               subject_trust_frameworks_supported=['ebsi'],                                                        
               id_token_types_supported=['subject_signed_id_token', 'attester_signed_id_token']                    
           )                                                                                                       

## 1.2 Perform authorization request and obtain ID token request

In [28]:
from ebsi_wallet.siop_auth.util import (
    perform_authorization, 
    AuthorizationRequestQueryParams,
    get_authorization_response_query_params,
    generate_code_challenge,
    generate_code_verifier
)
import uuid
import json

state = str(uuid.uuid4())
nonce = str(uuid.uuid4())
code_verifier = generate_code_verifier()

authorization_details = [{"type":"openid_credential","format":"jwt_vc","types":credential_offer.credentials[0].get("types"),"locations": ["https://api-conformance.ebsi.eu/conformance/v3/issuer-mock"]}]
redirect_uri = "http://localhost:8080"
client_metadata = {"vp_formats_supported":{"jwt_vp":{"alg":["ES256"]},"jwt_vc":{"alg":["ES256"]}},"response_types_supported":["vp_token","id_token"],"authorization_endpoint":redirect_uri}
authorization_request_query_params = AuthorizationRequestQueryParams(
    response_type="code",
    scope="openid",
    state=state,
    client_id=key_did._did,
    authorization_details=json.dumps(authorization_details, separators=(',', ':')),
    redirect_uri=redirect_uri,
    nonce=nonce,
    code_challenge=generate_code_challenge(code_verifier),
    code_challenge_method="S256",
    client_metadata=json.dumps(client_metadata, separators=(',', ':')),
    issuer_state=credential_offer.grants.get("authorization_code").get("issuer_state")
)

auth_resp = await perform_authorization(auth_server_configuration.authorization_endpoint, authorization_request_query_params)
auth_resp_uri = str(auth_resp).split("Location': '")[1].split("'")[0]
console.log("Authorization response URI: ", auth_resp_uri)
auth_resp_query_params = get_authorization_response_query_params(auth_resp_uri)
console.log("Authorization response query params: ", auth_resp_query_params)

[23:27:32] Authorization response URI:                                                             ]8;id=784715;file:///tmp/ipykernel_59965/1836098038.py\1836098038.py]8;;\:]8;id=260191;file:///tmp/ipykernel_59965/1836098038.py#34\34]8;;\
           http://localhost:8080?state=210d0664-ab6d-4840-b88b-f2c13ff17f44&client_id=https%3A%2F%                 
           2Fapi-conformance.ebsi.eu%2Fconformance%2Fv3%2Fauth-mock&redirect_uri=https%3A%2F%2Fapi                 
           -conformance.ebsi.eu%2Fconformance%2Fv3%2Fauth-mock%2Fdirect_post&response_type=id_toke                 
           n&response_mode=direct_post&scope=openid&nonce=d123074d-9d4f-4598-a9b7-04c56cbceea4&req                 
           uest_uri=https%3A%2F%2Fapi-conformance.ebsi.eu%2Fconformance%2Fv3%2Fauth-mock%2Frequest                 
           _uri%2F7f0d834b-6831-4e2e-ad9c-021007d60caf                                                             

           Authorization response query params:                                                    ]8;id=348514;file:///tmp/ipykernel_59965/1836098038.py\1836098038.py]8;;\:]8;id=129854;file:///tmp/ipykernel_59965/1836098038.py#36\36]8;;\
           AuthorizationResponseQueryParams(                                                                       
               state='210d0664-ab6d-4840-b88b-f2c13ff17f44',                                                       
               client_id='https://api-conformance.ebsi.eu/conformance/v3/auth-mock',                               
               redirect_uri='https://api-conformance.ebsi.eu/conformance/v3/auth-mock/direct_post'                 
           ,                                                                                                       
               response_type='id_token',                                                                           
               response_mode='direct_post',                                                                        
               scope='openid',                                                                                     
               nonce='d123074d-9d4f-4598-a9b7-04c56cbceea4',                                                       
               request_uri='https://api-conformance.ebsi.eu/conformance/v3/auth-mock/request_uri/7                 
           f0d834b-6831-4e2e-ad9c-021007d60caf'                                                                    
           )                                                                                                       

## 1.3 Send ID token

In [29]:
from ebsi_wallet.siop_auth.util import send_id_token_response

id_token = key_did.generate_id_token(auth_server_uri=auth_resp_query_params.client_id, nonce=auth_resp_query_params.nonce)
auth_code_response = await send_id_token_response(auth_resp_query_params.redirect_uri, id_token, auth_resp_query_params.state)
auth_code_response = str(auth_code_response).split("Location': '")[1].split("'")[0]
state = parse_query_string_parameters_from_url(auth_code_response).get("state")[0]
auth_code = parse_query_string_parameters_from_url(auth_code_response).get("code")[0]
console.log("Authorization code: ", auth_code)

[23:27:33] Authorization code:  85279e4a-406e-488b-9d7c-a7c9015ace5b                                ]8;id=821197;file:///tmp/ipykernel_59965/1546471797.py\1546471797.py]8;;\:]8;id=357546;file:///tmp/ipykernel_59965/1546471797.py#8\8]8;;\

## 1.4 Exchange code for access token

In [30]:
from ebsi_wallet.siop_auth.util import exchange_auth_code_for_access_token

token_uri = auth_resp_query_params.client_id + "/token"
access_token_response = await exchange_auth_code_for_access_token(token_uri, key_did._did, auth_code, code_verifier)
console.log("Access token response: ", access_token_response)

           Access token response:                                                                    ]8;id=425560;file:///tmp/ipykernel_59965/474231906.py\474231906.py]8;;\:]8;id=54670;file:///tmp/ipykernel_59965/474231906.py#5\5]8;;\
           AccessTokenResponse(                                                                                    
               access_token='eyJ0eXAiOiJKV1QiLCJhbGciOiJFUzI1NiIsImtpZCI6IkJKeTViQ1Mta1dGRUhmekczYWV               
           ZUWlGbmVOYnFDWHRkLVlaVmk3cmxpRGsifQ.eyJub25jZSI6ImViMTJjYmYwLTBhMmYtNGZmNy1hNDE4LWI0YmE3Y               
           jk3ZjZmMiIsImNsYWltcyI6eyJhdXRob3JpemF0aW9uX2RldGFpbHMiOlt7InR5cGUiOiJvcGVuaWRfY3JlZGVudG               
           lhbCIsImZvcm1hdCI6Imp3dF92YyIsImxvY2F0aW9ucyI6WyJodHRwczovL2FwaS1jb25mb3JtYW5jZS5lYnNpLmV               
           1L2NvbmZvcm1hbmNlL3YzL2lzc3Vlci1tb2NrIl0sInR5cGVzIjpbIlZlcmlmaWFibGVDcmVkZW50aWFsIiwiVmVy               
           aWZpYWJsZUF0dGVzdGF0aW9uIiwiQ1RXYWxsZXRDcm9zc0RlZmVycmVkIl19XSwiY19ub25jZSI6ImIyMjNlNzBlL               
           TdlYzYtNDYwNS04YzQ0LWFhMzEwODc4OWUxNyIsImNfbm9uY2VfZXhwaXJlc19pbiI6ODY0MDAsImNsaWVudF9pZC               
           I6ImRpZDprZXk6ejJkbXpEODFjZ1B4OFZraTdKYnV1TW1GWXJXUGdZb3l0eWtVWjNleXFodDFqOUticHRRd29hajJ               
           WUDFTNkFoem83UkVGQ1Q0TkJUUFlkUWluQ1piQ2N5b3FXS2k5UTJ1RVczNkROU1hoQ3dpWW5HejZCQVprenl0UUFF               
           QkU1Y1BpZENHbmFkSDRTc0xEYlNaZUcyU0VDaHJxdlFwZEs0TWs4SDMydnMzQjVnOFdyN2tjYyJ9LCJpc3MiOiJod               
           HRwczovL2FwaS1jb25mb3JtYW5jZS5lYnNpLmV1L2NvbmZvcm1hbmNlL3YzL2F1dGgtbW9jayIsImF1ZCI6WyJodH               
           RwczovL2FwaS1jb25mb3JtYW5jZS5lYnNpLmV1L2NvbmZvcm1hbmNlL3YzL2lzc3Vlci1tb2NrIl0sInN1YiI6ImR               
           pZDprZXk6ejJkbXpEODFjZ1B4OFZraTdKYnV1TW1GWXJXUGdZb3l0eWtVWjNleXFodDFqOUticHRRd29hajJWUDFT               
           NkFoem83UkVGQ1Q0TkJUUFlkUWluQ1piQ2N5b3FXS2k5UTJ1RVczNkROU1hoQ3dpWW5HejZCQVprenl0UUFFQkU1Y               
           1BpZENHbmFkSDRTc0xEYlNaZUcyU0VDaHJxdlFwZEs0TWs4SDMydnMzQjVnOFdyN2tjYyIsImlhdCI6MTY4NzE5Nz               
           Q1MywiZXhwIjoxNjg3MjgzODUzfQ._NxT90yYWL8gueYvJNd40LQOC6poxHu3fxip-OCFpTVdGjR5oIfQPVFC14A1               
           qxrznOBkEi0wXZVkOVLO6SniBA',                                                                            
               token_type='Bearer',                                                                                
               expires_in=86400,                                                                                   
               id_token='eyJ0eXAiOiJKV1QiLCJhbGciOiJFUzI1NiIsImtpZCI6IkJKeTViQ1Mta1dGRUhmekczYWVZUWl               
           GbmVOYnFDWHRkLVlaVmk3cmxpRGsifQ.eyJub25jZSI6IjIxYjk2MjY2LWE2OTYtNDQxNy04N2E1LTA0OGNmZjJkN               
           2NjNSIsImlzcyI6Imh0dHBzOi8vYXBpLWNvbmZvcm1hbmNlLmVic2kuZXUvY29uZm9ybWFuY2UvdjMvYXV0aC1tb2               
           NrIiwic3ViIjoiZGlkOmtleTp6MmRtekQ4MWNnUHg4VmtpN0pidXVNbUZZcldQZ1lveXR5a1VaM2V5cWh0MWo5S2J               
           wdFF3b2FqMlZQMVM2QWh6bzdSRUZDVDROQlRQWWRRaW5DWmJDY3lvcVdLaTlRMnVFVzM2RE5TWGhDd2lZbkd6NkJB               
           Wmt6eXRRQUVCRTVjUGlkQ0duYWRINFNzTERiU1plRzJTRUNocnF2UXBkSzRNazhIMzJ2czNCNWc4V3I3a2NjIiwiY               
           XVkIjoiZGlkOmtleTp6MmRtekQ4MWNnUHg4VmtpN0pidXVNbUZZcldQZ1lveXR5a1VaM2V5cWh0MWo5S2JwdFF3b2               
           FqMlZQMVM2QWh6bzdSRUZDVDROQlRQWWRRaW5DWmJDY3lvcVdLaTlRMnVFVzM2RE5TWGhDd2lZbkd6NkJBWmt6eXR               
           RQUVCRTVjUGlkQ0duYWRINFNzTERiU1plRzJTRUNocnF2UXBkSzRNazhIMzJ2czNCNWc4V3I3a2NjIiwiaWF0Ijox               
           Njg3MTk3NDUzLCJleHAiOjE2ODcxOTc1NzN9.X1mEcraG5F9zlpEcpVPAQguyRTfRaFEzPtkNQEStNcVjYx57VNNq               
           XqMR5IBsqYJQH-hdzSTcSUns9zhpNqLNUQ',                                                                    
               c_nonce='b223e70e-7ec6-4605-8c44-aa3108789e17',                                                     
               c_nonce_expires_in=86400      

## 1.5 Request credential to get acceptance token to initiate deferred flow 

In [31]:
from ebsi_wallet.siop_auth.util import send_credential_request

credential_request_jwt = key_did.generate_credential_request(credential_issuer_configuration.credential_issuer, access_token_response.c_nonce)
console.log("Credential request JWT: ", credential_request_jwt)
acceptance_token_response = await send_credential_request(credential_issuer_configuration.credential_endpoint, access_token_response.access_token, credential_request_jwt, credential_offer.credentials[0].get("types"))
console.log("Acceptance token response: ", acceptance_token_response)## 1.6 Request credential using deferred endpoint

           Credential request JWT:                                                                   ]8;id=684232;file:///tmp/ipykernel_59965/380321948.py\380321948.py]8;;\:]8;id=181587;file:///tmp/ipykernel_59965/380321948.py#4\4]8;;\
           eyJhbGciOiJFUzI1NiIsImtpZCI6ImRpZDprZXk6ejJkbXpEODFjZ1B4OFZraTdKYnV1TW1GWXJXUGdZb3l0eWtVW               
           jNleXFodDFqOUticHRRd29hajJWUDFTNkFoem83UkVGQ1Q0TkJUUFlkUWluQ1piQ2N5b3FXS2k5UTJ1RVczNkROU1               
           hoQ3dpWW5HejZCQVprenl0UUFFQkU1Y1BpZENHbmFkSDRTc0xEYlNaZUcyU0VDaHJxdlFwZEs0TWs4SDMydnMzQjV               
           nOFdyN2tjYyN6MmRtekQ4MWNnUHg4VmtpN0pidXVNbUZZcldQZ1lveXR5a1VaM2V5cWh0MWo5S2JwdFF3b2FqMlZQ               
           MVM2QWh6bzdSRUZDVDROQlRQWWRRaW5DWmJDY3lvcVdLaTlRMnVFVzM2RE5TWGhDd2lZbkd6NkJBWmt6eXRRQUVCR               
           TVjUGlkQ0duYWRINFNzTERiU1plRzJTRUNocnF2UXBkSzRNazhIMzJ2czNCNWc4V3I3a2NjIiwidHlwIjoib3Blbm               
           lkNHZjaS1wcm9vZitqd3QifQ.eyJhdWQiOiJodHRwczovL2FwaS1jb25mb3JtYW5jZS5lYnNpLmV1L2NvbmZvcm1h               
           bmNlL3YzL2lzc3Vlci1tb2NrIiwiZXhwIjoxNjg3MjgzODUzLCJpYXQiOjE2ODcxOTc0NTMsImlzcyI6ImRpZDprZ               
           Xk6ejJkbXpEODFjZ1B4OFZraTdKYnV1TW1GWXJXUGdZb3l0eWtVWjNleXFodDFqOUticHRRd29hajJWUDFTNkFoem               
           83UkVGQ1Q0TkJUUFlkUWluQ1piQ2N5b3FXS2k5UTJ1RVczNkROU1hoQ3dpWW5HejZCQVprenl0UUFFQkU1Y1BpZEN               
           HbmFkSDRTc0xEYlNaZUcyU0VDaHJxdlFwZEs0TWs4SDMydnMzQjVnOFdyN2tjYyIsIm5vbmNlIjoiYjIyM2U3MGUt               
           N2VjNi00NjA1LThjNDQtYWEzMTA4Nzg5ZTE3In0.-Q3FjJ2s1kmS6hrUWqTpiepjDOsviRhRQqACsThgDYKlRMn7r               
           JHusNyRpprfO-Av1JIGvUYEsoZpfDdUCpa1Bg                                                                   

[23:27:34] Acceptance token response:                                                                ]8;id=877268;file:///tmp/ipykernel_59965/380321948.py\380321948.py]8;;\:]8;id=454804;file:///tmp/ipykernel_59965/380321948.py#6\6]8;;\
           {'acceptance_token': 'Zpwk5Dtdfd8rSW3bwjszXetpW3bN1HC7t9fibOeY1Mo'}                                     

## 1.6 Request credential using deferred endpoint

In [32]:
from ebsi_wallet.siop_auth.util import send_deferred_credential_request
from time import sleep

console.log("Waiting for 5 seconds...")
sleep(5)
console.log("Sending deferred credential request...")
credential_response = await send_deferred_credential_request(credential_issuer_configuration.deferred_credential_endpoint, acceptance_token_response.get("acceptance_token"))
print("Credential response: ", credential_response)

           Waiting for 5 seconds...                                                                 ]8;id=42516;file:///tmp/ipykernel_59965/1677177840.py\1677177840.py]8;;\:]8;id=170431;file:///tmp/ipykernel_59965/1677177840.py#4\4]8;;\

[23:27:39] Sending deferred credential request...                                                   ]8;id=424704;file:///tmp/ipykernel_59965/1677177840.py\1677177840.py]8;;\:]8;id=220761;file:///tmp/ipykernel_59965/1677177840.py#6\6]8;;\

Credential response:  {'format': 'jwt_vc', 'credential': 'eyJhbGciOiJFUzI1NiIsInR5cCI6IkpXVCIsImtpZCI6ImRpZDplYnNpOnpoSkFSalBONjljRXRnUHhIZW4xTWlkI3VPa3Y1TE9pbzlYQnE2a1JRaHNaMHpOTVZ1czBjNFZqcFB0cDY1dTVjWXcifQ.eyJqdGkiOiJ2YzplYnNpOmNvbmZvcm1hbmNlI2VmOGU3ZWUxLTkyYWQtNGQxMy1hOGU5LTA3N2MyODk0OTRiZCIsInN1YiI6ImRpZDprZXk6ejJkbXpEODFjZ1B4OFZraTdKYnV1TW1GWXJXUGdZb3l0eWtVWjNleXFodDFqOUticHRRd29hajJWUDFTNkFoem83UkVGQ1Q0TkJUUFlkUWluQ1piQ2N5b3FXS2k5UTJ1RVczNkROU1hoQ3dpWW5HejZCQVprenl0UUFFQkU1Y1BpZENHbmFkSDRTc0xEYlNaZUcyU0VDaHJxdlFwZEs0TWs4SDMydnMzQjVnOFdyN2tjYyIsImlzcyI6ImRpZDplYnNpOnpoSkFSalBONjljRXRnUHhIZW4xTWlkIiwibmJmIjoxNjg3MTk3NDU0LCJleHAiOjE2ODcyODM4NTQsImlhdCI6MTY4NzE5NzQ1NCwidmMiOnsiQGNvbnRleHQiOlsiaHR0cHM6Ly93d3cudzMub3JnLzIwMTgvY3JlZGVudGlhbHMvdjEiXSwiaWQiOiJ2YzplYnNpOmNvbmZvcm1hbmNlI2VmOGU3ZWUxLTkyYWQtNGQxMy1hOGU5LTA3N2MyODk0OTRiZCIsInR5cGUiOlsiVmVyaWZpYWJsZUNyZWRlbnRpYWwiLCJWZXJpZmlhYmxlQXR0ZXN0YXRpb24iLCJDVFdhbGxldENyb3NzRGVmZXJyZWQiXSwiaXNzdWVyIjoiZGlkOmVic2k6emhKQVJqUE42OWNFdGdQeEh